<a href="https://colab.research.google.com/github/srivarshini-pratapaneni/srivarshini_INFO5731_spring2021/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [10]:

!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor

     |████████████████████████████████| 133kB 12.5MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=f388e69113b69c91c48dee9bc649148fe0e0f7f2a4aae9dac02f29a039ec8589
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 12.1MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306378 sha256=176e30889fb3c6672e13851016851f34d0ea0aa2b3ac31dce73d0bfeafb02478
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu
     |████████████████████████████████| 41.4MB 100kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834237 sha256=b41eb62e7a2e970bf48112a3fb7055fc1de73d6749e2efb8b974f029162e53c0
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a

In [11]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/srivarshini-pratapaneni/srivarshini_INFO5731_spring2021/main/sentimental-analysis.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [14]:
#preprocessing
from nltk.corpus import stopwords
from textblob import TextBlob
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Review'] = df['Review'].apply(lambda x: str(TextBlob(x).correct()))

In [15]:
#Extracting Words Related to Sentiment using Polyglot
from polyglot.text import Text
sentimental_words = []
for line in df['Review']:
  text = Text(line)
  for w in text.words:
    if w.polarity != 0:
      sentimental_words.append(w)
sentimental_words

['brilliance',
 'grotesque',
 'hard',
 'mesmerizing',
 'tragic',
 'funny',
 'rollercoaster',
 'proper',
 'best',
 'isolated',
 'sorry',
 'encourage',
 'better',
 'respect',
 'like',
 'isolated',
 'masterpiece',
 'best',
 'best',
 'best',
 'willingly',
 'remarkable',
 'disturbing',
 'violent',
 'proving',
 'corruption',
 'rich',
 'talented',
 'believable',
 'shake',
 'bitter',
 'comforting',
 'perfect',
 'fearless',
 'stunning',
 'impossible',
 'dark',
 'joke',
 'stunned',
 'accomplished',
 'impossible',
 'exceeds',
 'dark',
 'crime',
 'like',
 'falling',
 'joke',
 'happy',
 'ignore',
 'complaints',
 'pernicious',
 'embarrassing',
 'better',
 'criminal',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'romantic',
 'scars',
 'amazing',
 'best',
 'breathtaking',
 'uncomfortable',
 'like',
 'horrible',
 'great',
 'inspiration',
 'honest',
 'violent',
 'disturbing',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'madness',
 'enjoy',
 'masterpiece',
 'amazing',
 'enjoy',
 'jo

In [16]:
#Ranking Sentiment Words By Frequency
from collections import Counter
frequencies = Counter(sentimental_words)
rank_df = pd.DataFrame(list(frequencies.most_common()), columns=['Words', 'Frequencies'])
rank_df.index = list(range(1, len(frequencies.most_common())+1))
rank_df

,Words,Frequencies
1,joke,135
2,like,61
3,good,53
4,best,40
5,dark,33
...,...,...
467,cry,1
468,marvelous,1
469,tolerable,1
470,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [17]:
def Encoder(senti):
  sentiment_encoded = 0
  if senti == 'Positive':
    sentiment_encoded = 1
  elif senti == 'Negative':
    sentiment_encoded = 2
  return sentiment_encoded


In [18]:
actual_sentiment = []
for senti in df['Sentimental Analysis']:
  actual_sentiment.append(Encoder(senti))

In [19]:
#using textblob
from textblob import TextBlob
textblob_sentiments = []
for line in df['Review']:
  polarity = TextBlob(line).sentiment.polarity
  if polarity > 0:
    textblob_sentiments.append(1)
  elif polarity < 0:
    textblob_sentiments.append(2)
  elif polarity == 0.0:
    textblob_sentiments.append(0)
text_blob_df = pd.DataFrame(list(zip(df['Review'], actual_sentiment, textblob_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
text_blob_df

,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,2
1,movie felt alone isolated truly relate underst...,1,0
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,2
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,2
105,plan watching would suggest shipping last 20 3...,2,2
106,want title make wouldn know based comicbook ch...,1,1


In [20]:
from sklearn.metrics import f1_score, accuracy_score
text_blob_accuracy = accuracy_score(text_blob_df['Actual'], text_blob_df['Predicted'])*100
text_blob_f1_score = f1_score(text_blob_df['Actual'], text_blob_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0} and F1 Score is {1}".format(text_blob_accuracy, text_blob_f1_score))

Text Blob Accuracy is 56.481481481481474 and F1 Score is 0.399400871459695


In [21]:
#using vader
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_sentiments = []
for line in df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_sentiments.append(1)
  elif polarity < 0:
    vader_sentiments.append(2)
  elif polarity == 0.0:
    vader_sentiments.append(0)
vader_df = pd.DataFrame(list(zip(df['Review'], actual_sentiment, vader_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
vader_df

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,1
1,movie felt alone isolated truly relate underst...,1,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,1
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,1
105,plan watching would suggest shipping last 20 3...,2,1
106,want title make wouldn know based comicbook ch...,1,2


In [23]:
from sklearn.metrics import f1_score, accuracy_score
vader_accuracy = accuracy_score(vader_df['Actual'], vader_df['Predicted'])*100
vader_f1_score = f1_score(vader_df['Actual'], vader_df['Predicted'], average = 'macro')
print("Vader Accuracy is {0} and F1 Score is {1}".format(vader_accuracy, vader_f1_score))

Vader Accuracy is 58.333333333333336 and F1 Score is 0.3673289183222958


In [24]:
#using SVM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
TFid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [27]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(df['Review'],df['Sentimental Analysis'],test_size=0.33)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_X_vectors = TFid_vectorizer.fit_transform(Train_X)
Test_X_vectors = TFid_vectorizer.transform(Test_X)

In [28]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(Train_X_vectors, Train_Y)
predicted = svm_model.predict(Test_X_vectors)
report = classification_report(Test_Y, predicted, output_dict=True)
report

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.37499999999999994,
  'precision': 0.5,
  'recall': 0.3,
  'support': 10},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 3},
 '2': {'f1-score': 0.7547169811320754,
  'precision': 0.6666666666666666,
  'recall': 0.8695652173913043,
  'support': 23},
 'accuracy': 0.6388888888888888,
 'macro avg': {'f1-score': 0.3765723270440251,
  'precision': 0.38888888888888884,
  'recall': 0.3898550724637681,
  'support': 36},
 'weighted avg': {'f1-score': 0.5863469601677148,
  'precision': 0.5648148148148148,
  'recall': 0.6388888888888888,
  'support': 36}}

Analysis:
On running all the algorithms, i found highest accuray in SVM than vader and text blob. Hence, i conclude that SVM is more accurate comparetively. The accuracy details are:
SVM: 58.33
Vader: 58
Textblob: 56.3